### Synopsis
This code averages and reduces g2 for temperature ramping (up and down) of protein H04

In [1]:

import numpy as np
import matplotlib.pyplot as plt
import h5py
import pickle

import sys
sys.path.append('../common')
from utils import process_group

# mount_point = '/gdata/s8id-dmdtn/'
# fpath1 = mount_point + '2021-2/babnigg202107_2/cluster_results_QZ'
# fpath2 = mount_point + '2022-1/babnigg202203/cluster_results_QZ'

mount_point = "/home/8-id-i/2024-3/2024_1228_qz_llps_analysis/2025_08_analysis/"
fpath1 = mount_point + '2021-2/cluster_results_all'
fpath2 = mount_point + '2022-1/cluster_results_all'

coef_sam = 6.93e4  # Absolute scattering cross-section coefficient for the samples
coef_buf = 7.62e4  # Absolute scattering cross-section coefficient for the buffer


### Averaging of SA-XPCS results, temperature ramp up

In [2]:
avg_ramp_up, t_el, ql_dyn, ql_sta = process_group(group='T214',
                                              num_sections=20,
                                              prefix=fpath1, 
                                              zone_idx=1,
                                              num_cores=24)

index	 T-min(C)	 T-max(C)	 T-mean(C)	 points
n=00	    20.01	    21.07	  20.5428	 120
n=01	    21.08	    22.15	  21.6162	 120
n=02	    22.15	     23.2	  22.6776	 120
n=03	    23.21	    24.27	  23.7387	 120
n=04	    24.28	    25.35	   24.815	 120
n=05	    25.35	    26.41	  25.8803	 120
n=06	    26.42	    27.49	  26.9545	 120
n=07	    27.49	    28.56	  28.0278	 120
n=08	    28.58	    29.63	  29.0962	 120
n=09	    29.64	    30.71	    30.17	 119
n=10	    30.73	    31.77	  31.2451	 119
n=11	    31.77	    32.85	  32.3104	 119
n=12	    32.86	    33.91	  33.3849	 119
n=13	    33.92	    34.99	  34.4613	 119
n=14	     35.0	    36.05	  35.5318	 119
n=15	    36.07	    37.12	  36.5973	 119
n=16	    37.13	    38.22	  37.6797	 119
n=17	    38.23	    39.28	  38.7471	 119
n=18	    39.29	    40.36	  39.8242	 119
n=19	    40.37	    41.42	  40.8936	 119


### Averaging of SA-XPCS results, temperature ramp down

In [3]:
avg_ramp_down, t_el, ql_dyn, ql_sta = process_group(group='T215',
                                              num_sections=20,
                                              prefix=fpath1, 
                                              zone_idx=1,
                                              num_cores=24)

index	 T-min(C)	 T-max(C)	 T-mean(C)	 points
n=00	    40.33	     41.4	  40.8807	 120
n=01	    39.25	    40.33	  39.7944	 120
n=02	    38.19	    39.25	  38.7252	 120
n=03	    37.12	    38.19	  37.6515	 120
n=04	    36.02	    37.11	  36.5615	 120
n=05	    34.93	    36.01	  35.4707	 120
n=06	    33.83	    34.91	  34.3743	 120
n=07	    32.74	    33.81	  33.2795	 120
n=08	    31.64	    32.73	  32.1815	 120
n=09	    30.58	    31.63	  31.1047	 120
n=10	     29.5	    30.57	  30.0445	 119
n=11	    28.43	     29.5	   28.961	 119
n=12	    27.38	    28.43	  27.9027	 119
n=13	     26.3	    27.38	  26.8413	 119
n=14	    25.26	    26.29	  25.7751	 119
n=15	    24.19	    25.24	  24.7182	 119
n=16	    23.11	    24.18	   23.648	 119
n=17	    22.06	    23.11	   22.581	 119
n=18	    21.01	    22.05	  21.5256	 119
n=19	    19.99	    20.99	  20.4811	 119


### Averaging of background 

In [4]:

avg_bg, _, _, _ = process_group(group='D029',
                                 num_sections=1, 
                                 prefix=fpath1,
                                 zone_idx='auto',
                                 num_cores=24)


index	 T-min(C)	 T-max(C)	 T-mean(C)	 points
n=00	    24.98	    25.02	  25.0006	 200


### Data reduction

Subtracting I(Q) with background, converting to absolute scattering cross-section, and dump to pickle file for plotting

In [6]:


for ii in range(len(avg_ramp_up)):
    avg_ramp_up[ii]["saxs_1d"] = coef_sam*avg_ramp_up[ii]["saxs_1d"] - coef_buf*avg_bg[0]["saxs_1d"]

for ii in range(len(avg_ramp_down)):
    avg_ramp_down[ii]["saxs_1d"] = coef_sam*avg_ramp_down[ii]["saxs_1d"] - coef_buf*avg_bg[0]["saxs_1d"]

with open('checkpoint.pickle', 'wb') as f:  # 'wb' for binary write mode
    pickle.dump(avg_ramp_up, f)  # Dump the object to the file
    pickle.dump(avg_ramp_down, f)
    pickle.dump(t_el, f)
    pickle.dump(ql_sta, f)
    pickle.dump(ql_dyn, f)
    